<a href="https://colab.research.google.com/github/ammn23/recsystem/blob/main/recipe_recsystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix
#

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
recipes=pd.read_csv('/content/drive/MyDrive/recipes_w_search_terms.csv')

In [4]:
recipes['diabetic'] = recipes['tags'].apply(lambda x: 1 if 'diabetic' in x else 0)

In [5]:
recipes['id']=recipes['id'].astype(str)

In [6]:
recipes.dropna(subset=['name'], inplace=True)

In [7]:
recipes.dropna(subset=['description'], inplace=True)

In [8]:
recipes['dairy-free'] = recipes['tags'].apply(lambda x: 1 if 'dairy-free' in x else 0)
recipes['gluten-free'] = recipes['tags'].apply(lambda x: 1 if 'gluten-free' in x else 0)
recipes['low-carb'] = recipes['tags'].apply(lambda x: 1 if 'low-carb' in x else 0)
recipes['vegan'] = recipes['tags'].apply(lambda x: 1 if 'vegan' in x else 0)
recipes['vegetarian'] = recipes['tags'].apply(lambda x: 1 if 'vegetarian' in x else 0)
recipes['low-calorie'] = recipes['tags'].apply(lambda x: 1 if 'low-calorie' in x else 0)
recipes['easy'] = recipes['tags'].apply(lambda x: 1 if 'easy' in x else 0)
recipes['easy'] = recipes['tags'].apply(lambda x: 1 if 'beginner-cook' in x else 0)
recipes['easy'] = recipes['tags'].apply(lambda x: 1 if 'beginner-cook' in x else 0)



In [9]:

recipes['time'] = recipes['tags'].apply(lambda x: 15 if '15-minutes-or-less' in x else (30 if '30-minutes-or-less' in x else (60 if '60-minutes-or-less' in x else 100)))

In [10]:
def categorize_meal(search_terms, name):
    if 'breakfast' in search_terms or 'brunch' in search_terms:
        return 'breakfast'
    elif 'lunch' in search_terms or 'soup' in search_terms or 'chicken' in search_terms or 'stew' in search_terms or 'beef' in search_terms:
        return 'lunch'
    elif 'dinner' in search_terms or 'soup' in search_terms or 'chicken' in search_terms or 'stew' in search_terms or 'beef' in search_terms:
        return 'dinner'
    elif 'snack' in search_terms or 'appetizer' in search_terms:
        return 'snack'
    elif 'drink' in search_terms or 'drink' in name or 'Coffee' in name or 'Cocktail' in name:
        return 'drink'
    elif 'side' in search_terms or 'salad' in search_terms:
        return 'side'
    elif 'dessert' in search_terms:
        return 'dessert'
    elif 'bread' in search_terms:
        return 'bread'
    elif 'sauce' in search_terms or 'Sauce' in name:
        return 'sauce'
    else:
        return 'other'

recipes['meal'] = recipes.apply(lambda row: categorize_meal(row['search_terms'], row['name']), axis=1)

In [11]:
meals_to_keep = ['dinner', 'lunch', 'dessert', 'side', 'snack', 'breakfast']
recipes = recipes[recipes['meal'].isin(meals_to_keep)]

In [12]:
recipes=recipes[recipes['easy']==0]
recipes.drop('easy',axis=1,inplace=True)

In [13]:
recipes=recipes[recipes['time']!=100]

In [14]:
# Filter the dataframe
filtered_recipes = recipes[(recipes['meal'] == 'dinner') | (recipes['meal'] == 'lunch')]

# Sample 3000 recipes
sampled_recipes = filtered_recipes.sample(n=3000, random_state=42) # Set random_state for reproducibility


In [15]:
filtered_recipes = recipes[(recipes['meal'] == 'dessert') | (recipes['meal'] == 'side') | (recipes['meal'] == 'breakfast') | (recipes['meal'] == 'snack')]

sampled_recipes2 = filtered_recipes.sample(n=1000, random_state=42)


In [16]:
combined_recipes = pd.concat([sampled_recipes, sampled_recipes2])


In [17]:
recipes=combined_recipes

In [18]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [19]:
recipes.drop(['search_terms','tags'],axis=1,inplace=True)

In [102]:
term_mapping = {
    'chicken breast': 'chicken',
    'chicken thigh': 'chicken',
    'chicken drumstick': 'chicken',
    'chicken wing': 'chicken',
    'pork chop': 'pork',
    'pork loin': 'pork',
    'pork rib': 'pork',
    'beef steak': 'beef',
    'beef stew': 'beef',
    'ground beef': 'beef',
    'olive oil': 'oil',
    'vegetable oil': 'oil',
    'canola oil': 'oil',
    'vidalia onions':'onion',
    'grapeseed oil':'oil',
    'lean ground beef':'beef',
    'sea salt':'salt'
}

In [103]:
def normalize(ingredients,term_mapping,lemmatizer,stop_words):
    normalized=[]
    ingredients=ingredients.split(',')
    for ingredient in ingredients:
        print(ingredient)
        norm = ingredient.lower()
        norm = re.sub(r'[^\w\s]', '', norm)
        norm = re.sub(r'^\s+', '', norm)
        print(norm)
        norm = term_mapping.get(norm, norm)

        words = norm.split()
        words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]

        norm=' '.join(words)
        normalized.append(norm)
        print('this'+norm)

    return normalized

normalized= [normalize(ingredient,term_mapping,lemmatizer,stop_words) for ingredient in recipes['ingredients']]


Streaming output truncated to the last 5000 lines.
eggs
thisegg
 'nonfat milk'
nonfat milk
thisnonfat milk
 'nonfat sour cream'
nonfat sour cream
thisnonfat sour cream
 'egg whites'
egg whites
thisegg white
 'sugar'
sugar
thissugar
 'light cherry pie filling']
light cherry pie filling
thislight cherry pie filling
['olive oil'
olive oil
thisoil
 'onion'
onion
thisonion
 'mushrooms'
mushrooms
thismushroom
 'garlic cloves'
garlic cloves
thisgarlic clove
 'firm tofu'
firm tofu
thisfirm tofu
 'turmeric'
turmeric
thisturmeric
 'salt and pepper']
salt and pepper
thissalt pepper
['low-fat blueberry yogurt'
lowfat blueberry yogurt
thislowfat blueberry yogurt
 'apple juice'
apple juice
thisapple juice
 'fresh blueberries'
fresh blueberries
thisfresh blueberry
 'unsweetened frozen sliced peaches']
unsweetened frozen sliced peaches
thisunsweetened frozen sliced peach
['zucchini'
zucchini
thiszucchini
 'yellow squash'
yellow squash
thisyellow squash
 'carrot'
carrot
thiscarrot
 'onion'
onion
thison

In [82]:
print(normalized)

[['beef', 'enchilada sauce', 'fatfree refried bean', 'mexican blend cheese', 'flour tortilla', 'green chilies', 'onion', 'hot sauce'], ['milk', 'cream mushroom soup', 'egg', 'fine noodle', 'cooked turkey', 'soft breadcrumb', 'colbymonterey jack cheese', 'green pepper', 'butter', 'pimento pepper'], ['pork rib chop', 'salt 26 freshly ground black pepper', 'canola oil', 'unsalted butter'], ['tomato', 'green pepper', 'onion', 'garlic clove', 'tomato juice', 'cucumber', 'vinegar', 'olive oil', 'salt', 'pepper', 'crouton'], ['firm tofu', 'mushroom', 'shallot', 'carrot', 'onion', 'celery rib', 'tomato paste', 'red wine', 'vegetable stock', 'garlic clove', 'peanut oil', 'salt'], ['extralarge shrimp', 'coarse salt', 'lemon2c juice', 'olive oil', 'chorizo sausage', 'flat leaf parsley', 'cilantro leaf', 'garlic clove', 'roasted red pepper', 'fireroasted tomato', 'yellow onion', 'seedless cucumber', 'celery rib', 'crusty bread', 'hot sauce'], ['beef', 'onion', 'garlic clove', 'cheddar cheese', 'ja

In [90]:
pd.set_option('display.max_colwidth', None)

In [104]:
normalized=[' '.join(doc) for doc in normalized]
vectorizer=TfidfVectorizer()
ingr_vector=vectorizer.fit_transform(normalized)